In [310]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from statistics import mean

In [311]:
df = yf.download("^IXIC", start = "2010-01-01", end = "2023-05-01")
df1 = yf.download("^NSEI", start = "2010-01-01", end = "2023-05-01")
data = (df + df1 - df1).dropna()
data1 = (df1 + df - df).dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [312]:
#CALCULATING CORRELATION COEFFICIENT

sum_x = sum(data['Adj Close'])
sum_y = sum(data1['Adj Close'])
sum_xy = sum(data['Adj Close'] * data1['Adj Close'])
sum_x2= sum(data['Adj Close'] * data['Adj Close'])
sum_y2 = sum(data1['Adj Close'] * data1['Adj Close'])
n = len(data.index)
cor_cf = (n*sum_xy - sum_x*sum_y)/math.sqrt((n*sum_x2 - (sum_x)**2)*(n*sum_y2 - (sum_y)**2))
print("Correlation(NASDAQ[i],NSE[i]) =",cor_cf)

#CHECKING LEAD-LAG RELATIONSHIP

Nasdaq = data[1:]
Nse = data1[0:-1]
sum_x = sum(Nasdaq['Adj Close'])
sum_y = sum(Nse['Adj Close'])
sum_xy = sum(Nasdaq['Adj Close'].values * Nse['Adj Close'].values)
sum_x2= sum(Nasdaq['Adj Close'] * Nasdaq['Adj Close'])
sum_y2 = sum(Nse['Adj Close'] * Nse['Adj Close'])
n = len(Nasdaq.index)
cor_cf = (n*sum_xy - sum_x*sum_y)/math.sqrt((n*sum_x2 - (sum_x)**2)*(n*sum_y2 - (sum_y)**2))
print("Correlation(NASDAQ[i+1],NSE[i]) =", cor_cf)

Nasdaq = data[0:-1]
Nse = data1[1:]
sum_x = sum(Nasdaq['Adj Close'])
sum_y = sum(Nse['Adj Close'])
sum_xy = sum(Nasdaq['Adj Close'].values * Nse['Adj Close'].values)
sum_x2= sum(Nasdaq['Adj Close'] * Nasdaq['Adj Close'])
sum_y2 = sum(Nse['Adj Close'] * Nse['Adj Close'])
n = len(Nasdaq.index)
cor_cf = (n*sum_xy - sum_x*sum_y)/math.sqrt((n*sum_x2 - (sum_x)**2)*(n*sum_y2 - (sum_y)**2))
print("Correlation(NASDAQ[i],NSE[i+1]) =", cor_cf)

Correlation(NASDAQ[i],NSE[i]) = 0.9513138758576922
Correlation(NASDAQ[i+1],NSE[i]) = 0.9507737700328892
Correlation(NASDAQ[i],NSE[i+1]) = 0.95165035077158


In [313]:
def atr(high, low, close, n):
    tr = np.amax(np.vstack(((high - low).to_numpy(), (abs(high - close)).to_numpy(), (abs(low - close)).to_numpy())).T, axis=1)
    return pd.Series(tr).rolling(n).mean().to_numpy()

def MACD(df, n1, n2,n3):
    df[f'EMA{n1}'] = df['Close'].ewm(span = n1, adjust = False).mean()
    df[f'EMA{n2}'] = df['Close'].ewm(span = n2, adjust = False).mean()
    df['MACD'] = df[f'EMA{n1}'] - df[f'EMA{n2}']
    df['signal'] = df['MACD'].ewm(span = n3, adjust = False).mean()

def Keltner(df,n1,n2,n3):
    df['kcmb'] =  df['Close'].ewm(span = n1, adjust = False).mean()
    df['ATR'] = atr(df['High'], df['Low'], df['Close'], n2)
    df['kcub'] = df['kcmb'] + n3 * df['ATR']
    df['kclb'] = df['kcmb'] - n3 * df['ATR']
    
def Bollinger(data, n1, n2):
    MA = data.Close.rolling(window=n1).mean()
    SD = data.Close.rolling(window=n1).std()
    data['bbmb'] = MA
    data['bbub'] = MA + (n2 * SD) 
    data['bblb'] = MA - (n2 * SD)

def Sharperatio(PnL):
    return ((PnL.mean())/PnL.std())*math.sqrt(252)

def MaxDrawdown(array):
    max_dd = 0
    peak = array[0]
    for val in array:
        if val > peak:
            peak = val
        dd = (val / peak) - 1
        if dd < max_dd:
            max_dd = dd
    return max_dd

In [314]:
def Trade_BOLLINGER(data):
    buy, sell = [], []
    flag = 0
    for i in range(2, len(data)):
        if flag == 1 and data.Close.iloc[i] > data.bbub.iloc[i] and data.Close.iloc[i - 1] < data.bbub.iloc[i - 1]:
            sell.append(i)
            flag = 0
        elif flag == 0 and data.Close.iloc[i] < data.bblb.iloc[i] and data.Close.iloc[i - 1] > data.bblb.iloc[i - 1]:
            buy.append(i)
            flag = 1
    if len(sell) != 0 and len(buy) != 0:
        if sell[0] < buy[0]:
            sell.pop(0)
        if buy[-1] > sell[-1]:
            buy.pop(-1)
    return (buy, sell)

def Trade_KELTNER(data):
    buy, sell = [], []
    flag = 0
    for i in range(2, len(data)):
        if flag == 0 and data.Close.iloc[i] > data.kcub.iloc[i] and data.Close.iloc[i - 1] < data.kcub.iloc[i - 1]:
            buy.append(i)
            flag = 1
        elif flag == 1 and data.Close.iloc[i] < data.kclb.iloc[i] and data.Close.iloc[i - 1] > data.kclb.iloc[i - 1]:
            sell.append(i)
            flag = 0
    if len(sell) != 0 and len(buy) != 0:
        if sell[0] < buy[0]:
            sell.pop(0)
        if buy[-1] > sell[-1]:
            buy.pop(-1)
    return (buy, sell)

def Trade_MACD(data):
    buy, sell = [], []
    for i in range(2, len(data)):
        if data.MACD.iloc[i] > data.signal.iloc[i] and data.MACD.iloc[i - 1] < data.signal.iloc[i - 1]:
            buy.append(i)
        elif data.MACD.iloc[i] < data.signal.iloc[i] and data.MACD.iloc[i - 1] > data.signal.iloc[i - 1]:
            sell.append(i)
    if len(sell) != 0 and len(buy) != 0:
        if sell[0] < buy[0]:
            sell.pop(0)
        if buy[-1] > sell[-1]:
            buy.pop(-1)
    return (buy, sell)

In [315]:
#BOLLINGER TRADING STRATEGY BRUTE FORCE PARAMETER OPTIMISATION
for n1 in range(15,50):
    for n2 in [1, 1.5, 2, 2.5, 3]:
        
        Bollinger(data,n1,n2)
        buy, sell = Trade_BOLLINGER(data)
        if len(sell) == 0 or len(buy) == 0:
                continue;
        Realbuy = [i for i in buy]
        Realsell = [i for i in sell]
        BuyPrices = data.Close.iloc[Realbuy]
        SellPrices = data.Close.iloc[Realsell]

        capital = []
        capital.append(100)
        returns = []
        for i in range(len(BuyPrices)):
            returns.append((SellPrices[i]/BuyPrices[i]) - 1)
            capital.append(capital[-1] * (1 + returns[i]))

        if capital[-1] > 450:
            print(n1, n2, capital[-1])
            

15 2.5 522.4934372369988
17 2.5 455.7822613700037
20 2.5 494.87253719230574
22 2.5 450.87304113841753


In [317]:
#OPTIMISED DATA

Keltner(data, 16, 10, 3)
buy, sell = Trade_KELTNER(data)
Realbuy = [i for i in buy]
Realsell = [i for i in sell]
BuyPrices = data1.Close.iloc[Realbuy]
SellPrices = data1.Close.iloc[Realsell]
capital = []
capital.append(100)
returns = []
for i in range(len(BuyPrices)):
    returns.append((SellPrices[i]/BuyPrices[i]) - 1)
    capital.append(capital[-1] * (1 + returns[i]))
returns = np.array(returns)
print("Keltner Channel Results:-")
print(capital[-1] - 100, Sharperatio(returns), MaxDrawdown(capital))


Bollinger(data,15,2.5)
buy, sell = Trade_BOLLINGER(data)
Realbuy = [i for i in buy]
Realsell = [i for i in sell]
BuyPrices = data1.Close.iloc[Realbuy]
SellPrices = data1.Close.iloc[Realsell]
capital = []
capital.append(100)
returns = []
for i in range(len(BuyPrices)):
    returns.append((SellPrices[i]/BuyPrices[i]) - 1)
    capital.append(capital[-1] * (1 + returns[i]))
returns = np.array(returns)
print("Bollinger Results:-")
print(capital[-1] - 100, Sharperatio(returns), MaxDrawdown(capital))

MACD(data,10,29,7)
buy, sell = Trade_MACD(data)
Realbuy = [i for i in buy]
Realsell = [i  for i in sell]
BuyPrices = data1.Close.iloc[Realbuy]
SellPrices = data1.Close.iloc[Realsell]
capital = []
capital.append(100)
returns = []
for i in range(len(BuyPrices)):
    returns.append((SellPrices[i]/BuyPrices[i]) - 1)
    capital.append(capital[-1] * (1 + returns[i]))
returns = np.array(returns)
print("MACD Results:-")
print(capital[-1] - 100, Sharperatio(returns), MaxDrawdown(capital))


Keltner Channel Results:-
226.81547906442194 8.089629465833603 -0.09051421231600798
Bollinger Results:-
132.39877302964214 13.832240778710622 -0.03308659010084436
MACD Results:-
502.34198663119525 5.386132245689329 -0.10020381789538857
